Face Recognition System

In [ ]:
import cv2
import numpy as np
import pickle
from PIL import Image
import matplotlib.pyplot as plt

print("✅ Libraries imported successfully!")

# Check camera availability
try:
    cap = cv2.VideoCapture(0)
    camera_available = cap.isOpened()
    cap.release()
    print(f"📹 Camera available: {camera_available}")
except:
    camera_available = False
    print("📹 Camera not detected")

In [ ]:
class FaceRecognitionSystem:
    def __init__(self):
        """Initialize the Face Recognition System using OpenCV"""
        # Initialize face detector
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        # Initialize face recognizer
        self.face_recognizer = cv2.face.LBPHFaceRecognizer_create()
        
        # Storage for face data
        self.known_faces = []
        self.known_names = []
        self.known_labels = []
        self.label_to_name = {}
        self.name_to_label = {}
        self.is_trained = False
        self.next_label = 0
        
        print("🔥 Face Recognition System initialized with OpenCV!")
    
    def detect_faces(self, image):
        """Detect faces in an image using OpenCV"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 4, minSize=(30, 30))
        return faces, gray
    
    def add_person(self, image_path_or_array, name):
        """Add a person to the recognition database"""
        try:
            # Handle both file path and numpy array
            if isinstance(image_path_or_array, str):
                image = cv2.imread(image_path_or_array)
                if image is None:
                    print(f"❌ Could not load image: {image_path_or_array}")
                    return False
            else:
                image = image_path_or_array
            
            # Detect faces
            faces, gray = self.detect_faces(image)
            
            if len(faces) > 0:
                # Use the first face found
                (x, y, w, h) = faces[0]
                face_region = gray[y:y+h, x:x+w]
                face_resized = cv2.resize(face_region, (100, 100))
                
                # Assign label to name
                if name not in self.name_to_label:
                    self.name_to_label[name] = self.next_label
                    self.label_to_name[self.next_label] = name
                    self.next_label += 1
                
                label = self.name_to_label[name]
                
                # Store face data
                self.known_faces.append(face_resized)
                self.known_labels.append(label)
                self.known_names.append(name)
                
                print(f"✅ Added {name} to the database")
                return True
            else:
                print(f"❌ No face found in image for {name}")
                return False
        except Exception as e:
            print(f"❌ Error adding {name}: {str(e)}")
            return False
    
    def train_model(self):
        """Train the face recognition model"""
        if len(self.known_faces) < 2:
            print("❌ Need at least 2 face samples to train!")
            return False
        
        try:
            # Train the LBPH recognizer
            self.face_recognizer.train(self.known_faces, np.array(self.known_labels))
            self.is_trained = True
            print("✅ Model trained successfully!")
            return True
        except Exception as e:
            print(f"❌ Training failed: {str(e)}")
            return False
    
    def recognize_faces(self, image):
        """Recognize faces in an image"""
        faces, gray = self.detect_faces(image)
        face_names = []
        
        for (x, y, w, h) in faces:
            face_region = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face_region, (100, 100))
            
            if self.is_trained:
                # Predict using trained model
                label, confidence = self.face_recognizer.predict(face_resized)
                
                # Lower confidence means better match (opposite of face_recognition library)
                if confidence < 100:  # Threshold for recognition
                    name = self.label_to_name.get(label, "Unknown")
                    confidence_score = max(0, (100 - confidence) / 100)  # Convert to 0-1 scale
                else:
                    name = "Unknown"
                    confidence_score = 0
            else:
                name = "Unknown"
                confidence_score = 0
            
            face_names.append((name, confidence_score))
        
        return faces, face_names
    
    def draw_results(self, image, face_locations, face_names):
        """Draw rectangles and names on the image"""
        result_image = image.copy()
        
        for (x, y, w, h), (name, confidence) in zip(face_locations, face_names):
            # Choose color based on recognition
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            
            # Draw rectangle around face
            cv2.rectangle(result_image, (x, y), (x+w, y+h), color, 2)
            
            # Draw label
            label = f"{name} ({confidence:.2f})" if name != "Unknown" else "Unknown"
            cv2.rectangle(result_image, (x, y-35), (x+w, y), color, cv2.FILLED)
            cv2.putText(result_image, label, (x + 6, y - 6), 
                       cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)
        
        return result_image
    
    def save_database(self, filename="face_database.pkl"):
        """Save the face database"""
        if not self.is_trained:
            print("❌ No trained model to save!")
            return
        
        data = {
            'known_faces': self.known_faces,
            'known_labels': self.known_labels,
            'known_names': self.known_names,
            'label_to_name': self.label_to_name,
            'name_to_label': self.name_to_label,
            'next_label': self.next_label
        }
        
        # Save the trained model
        model_file = filename.replace('.pkl', '_model.yml')
        self.face_recognizer.save(model_file)
        
        # Save other data
        with open(filename, 'wb') as f:
            pickle.dump(data, f)
        print(f"💾 Database saved to {filename} and {model_file}")
    
    def load_database(self, filename="face_database.pkl"):
        """Load the face database"""
        try:
            # Load data
            with open(filename, 'rb') as f:
                data = pickle.load(f)
            
            self.known_faces = data['known_faces']
            self.known_labels = data['known_labels']
            self.known_names = data['known_names']
            self.label_to_name = data['label_to_name']
            self.name_to_label = data['name_to_label']
            self.next_label = data['next_label']
            
            # Load trained model
            model_file = filename.replace('.pkl', '_model.yml')
            self.face_recognizer.read(model_file)
            self.is_trained = True
            
            print(f"📂 Database loaded from {filename}")
            print(f"👥 Loaded {len(set(self.known_names))} people")
            return True
        except FileNotFoundError:
            print(f"❌ Database file {filename} not found")
            return False
        except Exception as e:
            print(f"❌ Error loading database: {str(e)}")
            return False

# Initialize the system
face_system = FaceRecognitionSystem()

In [ ]:
# Helper functions

def capture_person_photos(name, num_photos=5):
    """Capture photos from webcam for training"""
    if not camera_available:
        print("❌ Camera not available!")
        return []
    
    cap = cv2.VideoCapture(0)
    photos = []
    count = 0
    
    print(f"📸 Capturing {num_photos} photos for {name}")
    print("📋 Instructions:")
    print("- Look at the camera")
    print("- Press SPACE to take a photo")
    print("- Press ESC to finish early")
    
    while count < num_photos:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip frame for mirror effect
        frame = cv2.flip(frame, 1)
        
        # Detect faces in current frame
        faces, gray = face_system.detect_faces(frame)
        
        # Draw rectangles around detected faces
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Add instructions
        cv2.putText(frame, f"Capturing {name}: {count}/{num_photos}", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, "SPACE=Capture, ESC=Exit", 
                   (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Show face count
        cv2.putText(frame, f"Faces detected: {len(faces)}", 
                   (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        cv2.imshow('Capture Photos', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord(' ') and len(faces) > 0:  # Space to capture
            photos.append(frame.copy())
            count += 1
            print(f"📷 Photo {count} captured!")
        elif key == 27:  # ESC to exit
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"✅ Captured {len(photos)} photos for {name}")
    return photos

def add_person_from_photos(name, photos):
    """Add a person using multiple photos"""
    if not photos:
        print("❌ No photos provided!")
        return False
    
    success_count = 0
    for i, photo in enumerate(photos):
        if face_system.add_person(photo, name):
            success_count += 1
        
    if success_count > 0:
        print(f"✅ Added {success_count}/{len(photos)} photos for {name}")
        # Train the model after adding new person
        face_system.train_model()
        return True
    else:
        print(f"❌ Failed to add any photos for {name}")
        return False

def live_recognition():
    """Start live face recognition"""
    if not camera_available:
        print("❌ Camera not available!")
        return
    
    if not face_system.is_trained:
        print("❌ Model not trained! Add some people first and train the model.")
        return
    
    cap = cv2.VideoCapture(0)
    print("🔴 Live recognition started!")
    print("Press 'q' to quit, 's' to save frame")
    
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip frame for mirror effect
        frame = cv2.flip(frame, 1)
        
        # Recognize faces
        face_locations, face_names = face_system.recognize_faces(frame)
        
        # Draw results
        result_frame = face_system.draw_results(frame, face_locations, face_names)
        
        # Add instructions
        cv2.putText(result_frame, "Press 'q' to quit, 's' to save", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(result_frame, f"People in DB: {len(set(face_system.known_names))}", 
                   (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        cv2.imshow('Live Face Recognition', result_frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            filename = f"recognition_frame_{frame_count}.jpg"
            cv2.imwrite(filename, result_frame)
            print(f"💾 Frame saved as {filename}")
            frame_count += 1
    
    cap.release()
    cv2.destroyAllWindows()
    print("✅ Live recognition stopped!")

def test_with_image(image_path):
    """Test recognition with an image file"""
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"❌ Could not load image: {image_path}")
            return
        
        print(f"🧪 Testing recognition with {image_path}")
        
        # Recognize faces
        face_locations, face_names = face_system.recognize_faces(image)
        
        # Draw results
        result_image = face_system.draw_results(image, face_locations, face_names)
        
        # Display results
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title('Original Image')
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
        plt.title(f'Recognition Results ({len(face_locations)} faces)')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Print results
        for i, (name, confidence) in enumerate(face_names):
            print(f"Face {i+1}: {name} (Confidence: {confidence:.3f})")
        
    except Exception as e:
        print(f"❌ Error testing image: {str(e)}")

print("🛠️ Helper functions ready!")

In [ ]:
# Create sample data and test the system

def create_sample_faces():
    """Create sample face images for testing"""
    print("🎨 Creating sample face data...")
    
    # Create a simple face image for demonstration
    def create_face_image(brightness=150, name="Person"):
        # Create a base image
        img = np.full((300, 300, 3), brightness, dtype=np.uint8)
        
        # Add face-like features
        center = (150, 150)
        
        # Face outline (oval)
        cv2.ellipse(img, center, (80, 100), 0, 0, 360, (brightness-30, brightness-30, brightness-30), -1)
        
        # Eyes
        cv2.circle(img, (125, 130), 12, (0, 0, 0), -1)  # Left eye
        cv2.circle(img, (175, 130), 12, (0, 0, 0), -1)  # Right eye
        
        # Nose
        cv2.circle(img, (150, 160), 8, (brightness-50, brightness-50, brightness-50), -1)
        
        # Mouth
        cv2.ellipse(img, (150, 190), (25, 15), 0, 0, 180, (brightness-60, brightness-60, brightness-60), -1)
        
        return img
    
    # Create sample images
    samples = {}
    
    # Alice - bright face
    alice_img = create_face_image(180, "Alice")
    samples["Alice"] = alice_img
    
    # Bob - medium face  
    bob_img = create_face_image(140, "Bob")
    samples["Bob"] = bob_img
    
    # Charlie - darker face
    charlie_img = create_face_image(120, "Charlie")
    samples["Charlie"] = charlie_img
    
    return samples

# Create and display sample data
sample_faces = create_sample_faces()

# Display the sample faces
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Sample Face Data', fontsize=16)

for i, (name, img) in enumerate(sample_faces.items()):
    axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[i].set_title(f'{name}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print("✅ Sample faces created!")

In [ ]:
# Add sample people to the database and train

print("👥 Adding people to the face recognition database...")

# Add each sample face to the system
added_people = []
for name, img in sample_faces.items():
    success = face_system.add_person(img, name)
    if success:
        added_people.append(name)

if added_people:
    print(f"✅ Successfully added: {added_people}")
    
    # Train the model with the added faces
    print("🎯 Training the recognition model...")
    training_success = face_system.train_model()
    
    if training_success:
        print(f"✅ Model trained successfully!")
        print(f"📊 Database Status:")
        print(f"People in database: {len(set(face_system.known_names))}")
        print(f"Known people: {list(set(face_system.known_names))}")
        print(f"Total face samples: {len(face_system.known_faces)}")
        
        # Save the database
        face_system.save_database("sample_face_database.pkl")
    else:
        print("❌ Training failed!")
else:
    print("❌ Failed to add any people to the database")
    print("The sample faces might not be detected by OpenCV's face detector")
    print("Try using real photos or webcam captures instead")

In [ ]:
# Test the recognition system

def create_test_image():
    """Create a test image with multiple faces"""
    print("🧪 Creating test image with multiple people...")
    
    # Create a larger test image
    test_img = np.full((400, 600, 3), 200, dtype=np.uint8)
    
    # Add Alice's face (left side)
    alice_face = sample_faces["Alice"]
    alice_resized = cv2.resize(alice_face, (150, 150))
    test_img[50:200, 50:200] = alice_resized
    
    # Add Bob's face (right side) 
    bob_face = sample_faces["Bob"]
    bob_resized = cv2.resize(bob_face, (150, 150))
    test_img[50:200, 400:550] = bob_resized
    
    # Add an unknown face (bottom center)
    unknown_face = np.full((120, 120, 3), 160, dtype=np.uint8)
    cv2.circle(unknown_face, (60, 60), 40, (100, 100, 100), -1)  # Different face
    cv2.circle(unknown_face, (50, 50), 5, (255, 255, 255), -1)   # Eye
    cv2.circle(unknown_face, (70, 50), 5, (255, 255, 255), -1)   # Eye
    cv2.rectangle(unknown_face, (55, 70), (65, 80), (255, 255, 255), -1)  # Mouth
    
    test_img[250:370, 240:360] = unknown_face
    
    return test_img

# Create and test the image
test_image = create_test_image()

print("🔍 Running face recognition on test image...")

# Recognize faces in the test image
face_locations, face_names = face_system.recognize_faces(test_image)

# Draw results
result_image = face_system.draw_results(test_image, face_locations, face_names)

# Display results
plt.figure(figsize=(15, 8))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
plt.title('Test Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
plt.title(f'Recognition Results ({len(face_locations)} faces detected)')
plt.axis('off')

plt.tight_layout()
plt.show()

# Print detailed results
print(f"\n🎯 Recognition Results:")
print(f"Faces detected: {len(face_locations)}")

for i, (name, confidence) in enumerate(face_names):
    status = "✅ Recognized" if name != "Unknown" else "❓ Unknown"
    print(f"Face {i+1}: {name} (Confidence: {confidence:.3f}) {status}")

print(f"\n📈 System Performance:")
print(f"Database size: {len(set(face_system.known_names)) if face_system.known_names else 0} people")
print(f"Recognition accuracy: {len([n for n, c in face_names if n != 'Unknown'])}/{len(face_names)} faces recognized")

In [ ]:
# Usage Instructions and Live Demo

print("🚀 Face Recognition System Ready!")
print("\n📋 Available Functions:")
print("1. capture_person_photos(name, num_photos) - Capture photos from webcam")
print("2. add_person_from_photos(name, photos) - Add person using multiple photos")
print("3. face_system.add_person(image, name) - Add person from single image")
print("4. live_recognition() - Start live camera recognition")
print("5. test_with_image(path) - Test with image file")
print("6. face_system.save_database(filename) - Save face database")
print("7. face_system.load_database(filename) - Load face database")

print("\n💡 Quick Start Guide:")
print("For real people with camera:")
print("  photos = capture_person_photos('YourName', 5)")
print("  add_person_from_photos('YourName', photos)")
print("  live_recognition()")

print("\nFor image files:")
print("  face_system.add_person('path/to/photo.jpg', 'PersonName')")
print("  face_system.train_model()  # Train after adding people")
print("  test_with_image('path/to/test.jpg')")

print("\nExample workflow:")
print("  # Add multiple people")
print("  photos1 = capture_person_photos('Alice', 3)")
print("  add_person_from_photos('Alice', photos1)")
print("  photos2 = capture_person_photos('Bob', 3)")
print("  add_person_from_photos('Bob', photos2)")
print("  # Start recognition")
print("  live_recognition()")

print(f"\n📊 Current Status:")
print(f"✅ System initialized: True")
print(f"👥 People in database: {len(set(face_system.known_names)) if face_system.known_names else 0}")
print(f"🎯 Model trained: {face_system.is_trained}")
print(f"📹 Camera available: {camera_available}")
print(f"🚀 Ready for recognition: {face_system.is_trained}")

if camera_available and face_system.is_trained:
    print("\n🎬 Uncomment below to start live recognition:")
    print("# live_recognition()")
elif camera_available:
    print("\n📸 Uncomment below to add yourself to the database:")
    print("# photos = capture_person_photos('YourName', 5)")
    print("# add_person_from_photos('YourName', photos)")
    print("# live_recognition()")
else:
    print("\n📁 For image files, use:")
    print("# face_system.add_person('path/to/your/photo.jpg', 'YourName')")
    print("# face_system.train_model()")

photos = capture_person_photos('YourName', 5)
add_person_from_photos('YourName', photos) 
live_recognition()